In [3]:
from keras.models import model_from_json
import numpy as np
import tensorflow as tf
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.preprocessing import image



Using TensorFlow backend.


In [4]:
json_file = open('testeasycheck.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("testeasycheck.h5")
print("Loaded model from disk")

Loaded model from disk


In [5]:
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 798, 598, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 798, 598, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 399, 299, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 397, 297, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 397, 297, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 198, 148, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 196, 146, 64)     

In [9]:
img_pred=image.load_img('data/i62.jpg',target_size=(800,600))
img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred,axis=0)
rslt=loaded_model.predict(img_pred)

#NORMALISATION FORMULA:
#a+((x-A)(b-a)/(B-A))
#where a,b are the normalisation unit limit
#A,B are pure/impure paddy seed minimum and maximum numerical value predicted by neural network
#x is the value which we going to normalise.


if rslt[0][0] == 1:
    prediction = 'Given image is "PURE PADDY"'
    print(prediction)
    frslt=1+(((rslt-0.9818)*(100-1))/(1-0.9818))
    print('Percentage of Purity level is',frslt)
    
else:
    prediction = 'Given image is "IMPURE PADDY"'
    print(prediction)
    frslt=1+(((rslt-0.000566)*(100-1))/(0.999723-0.000566))
    print('Percentage of Impurity level is',frslt)


Given image is "IMPURE PADDY"
Percentage of Impurity level is [[0.9462824]]


In [13]:
batch_size=2
nb_test_samples=238
img_width,img_height=800,600
test_data_dir='data/test'
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
     test_data_dir,
     target_size=(img_width,img_height),
     batch_size=batch_size,
     class_mode='binary',shuffle=False)


Found 239 images belonging to 2 classes.


In [14]:
Y_pred = loaded_model.predict_generator(test_generator, steps=np.ceil(nb_test_samples // batch_size+1), verbose=1, workers=0)
y_pred=np.where(Y_pred>0.5, 1, 0)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

120/120 [==============================] - 48s 396ms/step
Confusion Matrix
[[ 57  58]
 [  0 124]]


In [15]:
print('Classification Report')
target_names = ['pure', 'impure']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

        pure       1.00      0.50      0.66       115
      impure       0.68      1.00      0.81       124

    accuracy                           0.76       239
   macro avg       0.84      0.75      0.74       239
weighted avg       0.83      0.76      0.74       239

